# PROCAT Experiments

In this notebook, we run experiments on the 10K set of real catalogues from the PROCAT dataset.

##### Imports

In [58]:
import ast
import copy
import json
import numpy as np
import os
import pandas as pd
import pickle
import random
import time

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Parameter
from torch.autograd import Variable

from matplotlib import pyplot as plt
from tqdm import tqdm

# sacred
from sacred import Experiment
from sacred.observers import MongoObserver

# text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from itertools import chain
import nltk  # had to also run nltk.download('punkt')

# visuals
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [59]:
# show full width tables
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [60]:
# custom utils
from utils import count_params, train_epochs
from data import test_model_custom, compare_solved_sort_unique, \
    get_single_kendall_tau, get_single_spearman_rho, get_batch_rank_correlation, get_batch_rank_correlation_and_perc_valid
from procat_utils import get_catalogs_with_full_info, show_predicted_catalog, show_correct_catalog, get_prediction_as_offers, print_predicted_catalog

In [61]:
# reload if saved changes in modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Config

In [62]:
config = dict()

In [63]:
# sacred experiment
config['ptr_mask'] = True
config['dataset_folder'] = 'PROCAT_mini'
config['dataset_name'] = config['dataset_folder']
config['task'] = 'real_catalogs'
config['experiment_name'] = config['task'] + '_' + config['dataset_folder'] + '_mask_' + str(config['ptr_mask'])
config['db_name'] = 'sacred'
config['db_url'] = 'localhost:27017'

##### Model

In [64]:
# architecture
config['ptr_word_emb_dim'] = 32
config['ptr_offer_emb_dim'] = 64
config['ptr_hid_dim'] = 64
config['ptr_offer_rnn_layers'] = 2
config['ptr_catalog_rnn_layers'] = 2
config['ptr_dropout_catalogs'] = 0.05
config['ptr_dropout_offers'] = 0.05
config['ptr_bidir_catalogs'] = True
config['ptr_bidir_offers'] = False  # not handled 2021 04

# training
config['batch_size'] = 64
config['learning_rate'] =  0.0001
config['num_epochs'] = 1

##### Dataset

In [65]:
# cv size
config['test_small_size'] = 300

# text preprocessing
config['vocab_max'] =  300000 # vocabulary cap (for tokens)
config['unknown_token'] = '?UNK?'
config['eos_token'] = '?EOS?'
config['pad_token'] = '?PAD?'
config['max_tokens'] = 30

# catalog preprocessing
config['page_break_token'] = '?PAGE_BREAK?'
config['page_break_priority'] = 0
config['max_offer_tokens_per_catalog'] = 200  # choose based on distributions, includes page-breaks
config['pad_not_real_offer'] = '?NOT_REAL_OFFER?'
config['pad_not_real_offer_priority'] = -1

##### Config paths

In [66]:
# input
config['path_in_df_offers_csv'] = './{}/offer_features.csv'.format(config['dataset_folder'])
config['path_in_df_sections_csv'] = './{}/section_features.csv'.format(config['dataset_folder'])
config['path_in_df_catalog_csv'] = './{}/catalog_features.csv'.format(config['dataset_folder'])

config['path_in_df_catalog_train_csv'] = './{}/catalog_train_set_features.csv'.format(config['dataset_folder'])
config['path_in_df_catalog_test_csv'] = './{}/catalog_test_set_features.csv'.format(config['dataset_folder'])

config['path_in_dictionary'] = './{}/dictionary.pickle'.format(config['dataset_folder'])
config['path_in_catalog_id_to_section_ids'] = './{}/catalog_to_sections.pickle'.format(config['dataset_folder'])

config['path_in_section_id_to_offer_ids'] = './{}/section_to_offers.pickle'.format(config['dataset_folder'])
config['path_in_section_id_to_offer_vectors'] = './{}/section_id_to_offer_vectors.pickle'.format(config['dataset_folder'])
config['path_in_section_id_to_section_num'] = './{}/section_to_number.pickle'.format(config['dataset_folder'])
config['path_in_section_id_to_offer_priorities'] = './{}/section_id_to_offer_priorities.pickle'.format(config['dataset_folder'])

config['path_in_offer_id_to_priority'] = './{}/offer_to_priority.pickle'.format(config['dataset_folder'])
config['path_in_offer_id_to_vector'] = './{}/offer_to_vector.pickle'.format(config['dataset_folder'])

config['path_in_np_X_train'] = './{}/X_train.npy'.format(config['dataset_folder'])
config['path_in_np_Y_train'] = './{}/Y_train.npy'.format(config['dataset_folder'])
config['path_in_np_X_test'] = './{}/X_test.npy'.format(config['dataset_folder'])
config['path_in_np_Y_test'] = './{}/Y_test.npy'.format(config['dataset_folder'])

config['path_in_torch_X_train'] = './{}/X_train.pb'.format(config['dataset_folder'])
config['path_in_torch_Y_train'] = './{}/Y_train.pb'.format(config['dataset_folder'])
config['path_in_torch_X_test'] = './{}/X_test.pb'.format(config['dataset_folder'])
config['path_in_torch_Y_test'] = './{}/Y_test.pb'.format(config['dataset_folder'])

##### Training and Model

In [67]:
config

{'ptr_mask': True,
 'dataset_folder': 'PROCAT_mini',
 'dataset_name': 'PROCAT_mini',
 'task': 'real_catalogs',
 'experiment_name': 'real_catalogs_PROCAT_mini_mask_True',
 'db_name': 'sacred',
 'db_url': 'localhost:27017',
 'ptr_word_emb_dim': 32,
 'ptr_offer_emb_dim': 64,
 'ptr_hid_dim': 64,
 'ptr_offer_rnn_layers': 2,
 'ptr_catalog_rnn_layers': 2,
 'ptr_dropout_catalogs': 0.05,
 'ptr_dropout_offers': 0.05,
 'ptr_bidir_catalogs': True,
 'ptr_bidir_offers': False,
 'batch_size': 64,
 'learning_rate': 0.0001,
 'num_epochs': 1,
 'test_small_size': 300,
 'vocab_max': 300000,
 'unknown_token': '?UNK?',
 'eos_token': '?EOS?',
 'pad_token': '?PAD?',
 'max_tokens': 30,
 'page_break_token': '?PAGE_BREAK?',
 'page_break_priority': 0,
 'max_offer_tokens_per_catalog': 200,
 'pad_not_real_offer': '?NOT_REAL_OFFER?',
 'pad_not_real_offer_priority': -1,
 'path_in_df_offers_csv': './PROCAT_mini/offer_features.csv',
 'path_in_df_sections_csv': './PROCAT_mini/section_features.csv',
 'path_in_df_catalog_

## Experiment Tracking via Sacred

In [68]:
# initiate the experiment
ex = Experiment(name=config['experiment_name'], interactive=True)

# add an observer, storing experiment info
ex.observers.append(MongoObserver(url=config['db_url'], db_name=config['db_name']))

In [69]:
# experiment config
@ex.config
def ex_config():
    
    # general
    learning_rate = config['learning_rate']
    dataset = config['dataset_name'] 
    epochs = config['num_epochs']
    cfg = config
    
    # model
    model = None
    final_training_loss = None
    optimizer = None
    
    # db 
    db_name = config['db_name']
    db_url = config['db_url']

In [70]:
# experiment run function
@ex.main
def run_model_tests():

    # run all tests, get all results
    result_general, result_tau, result_spearman, rank_valid_perc = run_tests(current_tested_model, test_dataloader, config)
    
    # log params and results
    log_experiment_results(result_general, result_tau, result_spearman, rank_valid_perc)
    
    return round(result_general, 5)

In [71]:
def run_tests(a_model, a_dataloader, a_config):
    
    print('Model: ', a_model.__class__)

    a_model.eval() 
    
    # general accuracy
    result_general, _ = test_model_custom(a_model, a_dataloader, compare_solved_sort_unique, print_every=999999, x_name=0, y_name=1)
    print('Result: {:.4f}'.format(result_general))
    
    result_tau, rank_valid_perc = get_batch_rank_correlation_and_perc_valid(a_dataloader, a_model, get_single_kendall_tau, print_every=999999)
    print('K-Tau: {:.4f}, perc_valid: {}'.format(result_tau, rank_valid_perc))

    result_spearman, rank_valid_perc = get_batch_rank_correlation_and_perc_valid(a_dataloader, a_model, get_single_spearman_rho, print_every=999999)
    print('S-Rho: {:.4f}, perc_valid: {}'.format(result_spearman, rank_valid_perc))

    a_model.train()
    
    return result_general, result_tau, result_spearman, rank_valid_perc

In [72]:
def log_experiment_results(r_general, r_tau, r_spearman, rank_valid_perc):
    
    # round if not none
    if r_tau:
        r_tau = round(r_tau, 5)
    if r_spearman:
        r_spearman = round(r_spearman, 5)
    
    # log the results
    ex.log_scalar('test.general', r_general)
    ex.log_scalar('test.rank_correlation_valid_perc', rank_valid_perc)
    ex.log_scalar('test.tau', r_tau, 5)
    ex.log_scalar('test.rho', r_spearman, 5)

### Custom utils

In [73]:
def recreate_unshuffled_x(x, y):
    r = [x[i] for i in y]
    return r

In [74]:
random.seed(220788)

In [75]:
def get_example(a_dataloader, x_name=0, y_name=1):
    """
    Take a torch dataloader and get a single example.
    """
    a_batch = next(iter(a_dataloader))
    example_points = a_batch[x_name][0]
    example_solution = a_batch[y_name][0]

    return example_points, example_solution


def get_batch(a_dataloader, x_name=0, y_name=1):
    """
    Get a batch of points and solutions from a torch dataloader.
    """
    a_batch = next(iter(a_dataloader))
    batch_points = a_batch[x_name]
    batch_solutions = a_batch[y_name]

    return batch_points, batch_solutions


# Data
Load all needed data.

## Reload Data
Including:
- **SIGIR csv's**
    - df_offers
    - df_sections
    - df_catalogs
- **any meta-resources**:
    - (word2idx)
    - section to offer ids etc?
- **numpy X and Y**
    - train and test separately
- **torch dataloaders**
    - the actual torch dataloaders (for myself?

#### Load CSVs

In [76]:
df = pd.read_csv(config['path_in_df_offers_csv'], sep=';')
df.head(1)

,catalog_id,section,offer_id,priority,heading,description,text,text_tokenized,token_length,offer_as_vector
0,0003leq,1,fed3AqfY,2,Påskebryg,"33 cl. KYLLE - KYLLE. 3 33. + PANT. Sælges fra fredag d. 17/3. Ved køb af mere end 60 stk., er prisen herefter 4,98 kr","Påskebryg 33 cl. KYLLE - KYLLE. 3 33. + PANT. Sælges fra fredag d. 17/3. Ved køb af mere end 60 stk., er prisen herefter 4,98 kr","['påskebryg', '33', 'cl', '.', 'kylle', '-', 'kylle', '.', '3', '33', '.', '+', 'pant', '.', 'sælges', 'fra', 'fredag', 'd.', '17/3', '.', 'ved', 'køb', 'af', 'mere', 'end', '60', 'stk.', ',', 'er', '?EOS?']",33,"[11665, 315, 67, 5, 7298, 25, 7298, 5, 37, 315, 5, 130, 209, 5, 149, 56, 1172, 230, 11666, 5, 79, 184, 22, 200, 210, 82, 3012, 6, 31, 0]"


In [77]:
len(df)

10001

In [78]:
df_sections = pd.read_csv(config['path_in_df_sections_csv'], sep=';')
df_sections.head(1)

,catalog_id,section_id,section_num,offer_ids,offer_vectors,offer_priorities
0,0003leq,0003leq_1,1,"['fed3AqfY', '799dzqfY', 'b8a3DqfY', 'a4c88qfY', '3cf8qqfY', '88a5UqfY', '6d6e0qfY', 'ef59YqfY', 'ab38CqfY', '0cbffqfY', '0a4eEqfY']","[[11665, 315, 67, 5, 7298, 25, 7298, 5, 37, 315, 5, 130, 209, 5, 149, 56, 1172, 230, 11666, 5, 79, 184, 22, 200, 210, 82, 3012, 6, 31, 0], [488, 118, 1653, 16, 5, 28, 5, 38, 5, 188, 131, 56, 11667, 14, 1173, 8, 3604, 8, 7300, 8, 11668, 8, 11669, 0, 1, 1, 1, 1, 1, 1], [11670, 7301, 5, 1037, 30, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1038, 801, 1522, 3605, 2283, 2023, 29, 5, 8, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1654, 15, 5539, 5540, 11671, 30, 5, 19, 26, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1655, 11672, 5, 19, 26, 75, 30, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1304, 4351, 30, 5, 8, 140, 8, 477, 5, 1806, 5, 432, 99, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3013, 2599, 78, 29, 5, 8, 2284, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1230, 304, 8, 838, 5, 90, 30, 5, 79, 184, 22, 200, 210, 46, 457, 6, 31, 196, 370, 3014, 126, 5, 86, 31, 579, 323, 0, 1, 1, 1], [3015, 4352, 30, 5, 25, 168, 25, 727, 5, 728, 579, 5, 8, 1231, 8, 1656, 8, 2024, 3016, 8, 236, 9, 695, 8, 3606, 8, 409, 1523, 8, 0], [3017, 79, 184, 22, 200, 210, 78, 457, 6, 31, 196, 370, 11673, 126, 5, 125, 30, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]","[2, 2, 1, 1, 2, 1, 1, 1, 3, 3, 2]"


In [79]:
len(df_sections)

1483

In [80]:
df_catalogs = pd.read_csv(config['path_in_df_catalog_csv'], sep=';')
df_catalogs.head(1)

catalog_id                                                                                  section_ids                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             offer_ids_with_pb                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [81]:
len(df_catalogs)

65

In [82]:
# reload feature csv's split into train and test
df_catalogs_train = pd.read_csv(config['path_in_df_catalog_train_csv'], sep=';')
# df_catalogs_train.head(1)

In [83]:
df_catalogs_test = pd.read_csv(config['path_in_df_catalog_test_csv'], sep=';')
# df_catalogs_test.head(1)

In [84]:
len(df_catalogs_train), len(df_catalogs_test)

(52, 12)

#### Load Meta Resources (dicts)

In [85]:
# read word2idx dictionary
with open(config['path_in_dictionary'], 'rb') as f:
    word2idx = pickle.load(f)
list(word2idx.items())[:5]

[('?EOS?', 0),
 ('?PAD?', 1),
 ('?UNK?', 2),
 ('?PAGE_BREAK?', 3),
 ('?NOT_REAL_OFFER?', 4)]

In [86]:
# read catalog id to section ids
with open(config['path_in_catalog_id_to_section_ids'], 'rb') as f:
    catalog_id_to_section_ids = pickle.load(f)
print(list(catalog_id_to_section_ids.items())[:2])

[('0003leq', {'0003leq_4', '0003leq_1', '0003leq_6', '0003leq_3', '0003leq_7', '0003leq_2', '0003leq_5'}), ('0007YiY', {'0007YiY_16', '0007YiY_6', '0007YiY_10', '0007YiY_2', '0007YiY_13', '0007YiY_5', '0007YiY_12', '0007YiY_3', '0007YiY_8', '0007YiY_4', '0007YiY_9', '0007YiY_14', '0007YiY_7', '0007YiY_11', '0007YiY_15', '0007YiY_1'})]


In [87]:
# read section maps
with open(config['path_in_section_id_to_offer_ids'], 'rb') as f:
    section_id_to_offer_ids = pickle.load(f)
print(list(section_id_to_offer_ids.items())[:2])

[('0003leq_1', ['fed3AqfY', '799dzqfY', 'b8a3DqfY', 'a4c88qfY', '3cf8qqfY', '88a5UqfY', '6d6e0qfY', 'ef59YqfY', 'ab38CqfY', '0cbffqfY', '0a4eEqfY']), ('0003leq_2', ['0ef3QqfY', 'a3abpqfY', 'ce94wqfY', 'e545VqfY', '11aaxqfY', 'e4e12qfY', '431baqfY', '3030ZqfY', 'db3fNqfY', 'd638dqfY', '539csqfY', 'b25flqfY', '18cdFqfY', '50b5vqfY'])]


In [88]:
with open(config['path_in_section_id_to_offer_vectors'], 'rb') as f:
    section_id_to_offers_as_vectors = pickle.load(f)
print(list(section_id_to_offers_as_vectors.items())[0])

('0003leq_1', [[11665, 315, 67, 5, 7298, 25, 7298, 5, 37, 315, 5, 130, 209, 5, 149, 56, 1172, 230, 11666, 5, 79, 184, 22, 200, 210, 82, 3012, 6, 31, 0], [488, 118, 1653, 16, 5, 28, 5, 38, 5, 188, 131, 56, 11667, 14, 1173, 8, 3604, 8, 7300, 8, 11668, 8, 11669, 0, 1, 1, 1, 1, 1, 1], [11670, 7301, 5, 1037, 30, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1038, 801, 1522, 3605, 2283, 2023, 29, 5, 8, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1654, 15, 5539, 5540, 11671, 30, 5, 19, 26, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1655, 11672, 5, 19, 26, 75, 30, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1304, 4351, 30, 5, 8, 140, 8, 477, 5, 1806, 5, 432, 99, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [3013, 2599, 78, 29, 5, 8, 2284, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1230, 304, 8, 838, 5, 90, 30, 5, 79, 184, 22, 200, 210, 46, 457, 6, 31, 196, 370

In [89]:
with open(config['path_in_section_id_to_section_num'], 'rb') as f:
    section_id_to_section_num = pickle.load(f)
print(list(section_id_to_section_num.items())[:5])

[('0003leq_1', 1), ('0003leq_2', 2), ('0003leq_3', 3), ('0003leq_4', 4), ('0003leq_5', 5)]


In [90]:
with open(config['path_in_section_id_to_offer_priorities'], 'rb') as f:
    section_ids_to_offers_as_priorities = pickle.load(f)
print(list(section_ids_to_offers_as_priorities.items())[:2])

[('0003leq_1', [2, 2, 1, 1, 2, 1, 1, 1, 3, 3, 2]), ('0003leq_2', [3, 3, 1, 2, 1, 1, 2, 1, 3, 3, 1, 1, 1, 1])]


In [91]:
# read offer maps
with open(config['path_in_offer_id_to_priority'], 'rb') as f:
    offer_id_to_priority = pickle.load(f)
print(list(offer_id_to_priority.items())[:2])

[('fed3AqfY', 2), ('799dzqfY', 2)]


In [92]:
with open(config['path_in_offer_id_to_vector'], 'rb') as f:
    offer_id_to_vector = pickle.load(f)
print(list(offer_id_to_vector.items())[:2])

[('fed3AqfY', [11665, 315, 67, 5, 7298, 25, 7298, 5, 37, 315, 5, 130, 209, 5, 149, 56, 1172, 230, 11666, 5, 79, 184, 22, 200, 210, 82, 3012, 6, 31, 0]), ('799dzqfY', [488, 118, 1653, 16, 5, 28, 5, 38, 5, 188, 131, 56, 11667, 14, 1173, 8, 3604, 8, 7300, 8, 11668, 8, 11669, 0, 1, 1, 1, 1, 1, 1])]


#### Load Numpy Matrices (X & Y)

In [93]:
# train set
X_train = np.load(config['path_in_np_X_train'])
Y_train = np.load(config['path_in_np_Y_train'])
X_train.shape, Y_train.shape, X_train[0], Y_train[0]

((52, 200, 30), (52, 200), array([[   4,    4,    4, ...,    4,    4,    4],
        [   4,    4,    4, ...,    4,    4,    4],
        [   4,    4,    4, ...,    4,    4,    4],
        ...,
        [   4,    4,    4, ...,    4,    4,    4],
        [1413,   19,   26, ...,    1,    1,    1],
        [1660,  530, 1107, ...,    1,    1,    1]], dtype=int32), array([ 14,  71,  78,  75, 116,  16,  79, 121, 148, 101,  49, 174,  13,
         53,  62, 155,  65, 159, 167,   6,  94, 114,  18, 120,  32,  12,
        166, 150,  44, 165, 183,  63, 198,   8,  98,  23,  84, 151, 192,
        163,  72,  15,  47, 189, 129,  99,  97,  73,  34, 117, 125, 135,
         38,  19,  95, 164,  83,  92, 102,  37, 170, 199, 122, 188, 112,
        186, 157, 138, 195, 184, 179, 132, 145, 144, 181,  36, 141, 178,
         20, 142, 115, 175,  77, 130,  24, 128,  21, 113,  70,  27, 152,
         29,  96,  93, 173,  60,  42,  86, 194,  30,  39,  68,  57, 131,
          5, 127,  22,  76,  59, 197, 149,  11,  52,  74,

In [94]:
# test set
X_test = np.load(config['path_in_np_X_test'])
Y_test = np.load(config['path_in_np_Y_test'])
X_test.shape, Y_test.shape, X_test[0], Y_test[0]

((12, 200, 30),
 (12, 200),
 array([[ 1804,   255,     9, ...,  5512,    12,     0],
        [ 7715,  1113,    95, ...,     1,     1,     1],
        [    4,     4,     4, ...,     4,     4,     4],
        ...,
        [  693,  1257, 24201, ...,     1,     1,     1],
        [ 1761,  1135, 24343, ...,     1,     1,     1],
        [24115, 24116, 24117, ...,     1,     1,     1]], dtype=int32),
 array([ 15, 115,  90, 187, 167,  14, 163, 109, 178, 139,  85,  51,  45,
         16, 155,  54, 175,  86, 161,  78,  91,  47, 122, 162, 121,  74,
        137,  21,  37,  73,  53, 191,  32,  97,  69, 185,  10,   0, 143,
        180, 120,   5,  77, 182,  94,  75,  60, 157, 101,  83,  33, 184,
         11, 106, 197,  88, 144,  79, 131, 104, 193, 152, 117,  38, 105,
          7, 113,  57, 145,  48, 147,  43, 160,  71, 132, 196, 159, 138,
        127, 118, 135,  99, 110,  22,  44,  64,  62, 148,  36, 116,  34,
        119,  27,  23,  58, 174, 124,  56,  68, 181,  82, 142, 166,  84,
        188, 169, 

#### Load Torch Tensors (X & Y)

In [95]:
# train set
X_train_torch = torch.load(config['path_in_torch_X_train'])
Y_train_torch = torch.load(config['path_in_torch_Y_train'])
X_train_torch.size(), Y_train_torch.size()

(torch.Size([52, 200, 30]), torch.Size([52, 200]))

In [96]:
# full test set
X_test_full_torch = torch.load(config['path_in_torch_X_test'])
Y_test_full_torch = torch.load(config['path_in_torch_Y_test'])
X_test_full_torch.size(), Y_test_full_torch.size()

(torch.Size([12, 200, 30]), torch.Size([12, 200]))

In [97]:
# Y needs to be a long
Y_train_torch = Y_train_torch.long()
Y_test_full_torch = Y_test_full_torch.long()

In [98]:
Y_train_torch.type(), Y_test_full_torch.type()

('torch.LongTensor', 'torch.LongTensor')

#### Smaller test set
For validation here, due to memory size problems.

In [99]:
config['test_small_size']

300

In [100]:
# split
X_test_torch = X_test_full_torch[:config['test_small_size']]
Y_test_torch = Y_test_full_torch[:config['test_small_size']]
X_test_torch.size(), Y_test_torch.size()

(torch.Size([12, 200, 30]), torch.Size([12, 200]))

#### Turn into Torch Datasets and Dataloaders

In [101]:
# into datasets
dataset_train = TensorDataset(X_train_torch, Y_train_torch)
dataset_test = TensorDataset(X_test_torch, Y_test_torch)

In [102]:
# dataloaders
train_dataloader = DataLoader(dataset_train, batch_size=config['batch_size'], shuffle=True, num_workers=4)
test_dataloader = DataLoader(dataset_test, batch_size=config['batch_size'], shuffle=True, num_workers=4)

# Models

## Model PtrNet

In [103]:
from procat_models import PointerCatalogNetwork, \
    PointerEncoder, PointerAttention, PointerDecoder, PointerOfferEmbedder

In [104]:
# instantiate the model
model_ptrnet = PointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_ptrnet.float()

PointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (encoder): PointerEncoder(
    (lstm): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.05, bidirectional=True)
  )
  (decoder): PointerDecoder(
    (input_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_out): Linear(in_features=128, out_features=64, bias=True)
    (att): PointerAttention(
      (input_linear): Linear(in_features=64, out_features=64, bias=True)
      (context_linear): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (soft): Softmax(dim=1)
    )
  )
)

In [105]:
# how many params
count_params(model_ptrnet)

The model has 981,280 trainable parameters


In [106]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [107]:
# predict
_ = model_ptrnet(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [108]:
print(_[1].type())

torch.LongTensor


In [109]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [110]:
# optimizer
optimizer_ptrnet = optim.Adam(
    filter(lambda p: p.requires_grad, model_ptrnet.parameters()),
    lr=config['learning_rate'])

### Model Training | PtrNet

In [111]:
# train
last_loss_ptrnet = train_epochs(
    model_ptrnet,
    optimizer_ptrnet,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

Epoch 1 / 1: 100%|██████████| 1/1 [00:15<00:00, 15.06s/ batches, avg_loss=5.29881]


In [112]:
last_loss_ptrnet.item()

5.298806190490723

### Model Testing | PtrNet
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [113]:
model_ptrnet.eval()  # if we had dropout or batchnorm, we must do this

PointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (encoder): PointerEncoder(
    (lstm): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.05, bidirectional=True)
  )
  (decoder): PointerDecoder(
    (input_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_to_hidden): Linear(in_features=64, out_features=256, bias=True)
    (hidden_out): Linear(in_features=128, out_features=64, bias=True)
    (att): PointerAttention(
      (input_linear): Linear(in_features=64, out_features=64, bias=True)
      (context_linear): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (soft): Softmax(dim=1)
    )
  )
)

#### Sacred Experiment | Model PtrNet
We'll want to track the dataset, model params and the result of tests.

In [114]:
# experiment config
current_tested_model = model_ptrnet
current_optimizer = optimizer_ptrnet
current_last_loss = last_loss_ptrnet

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mini_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mini_mask_True - Started run with ID "347"


Model:  <class 'procat_models.PointerCatalogNetwork'>
Result: 0.0021
K-Tau: 0.0115, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mini_mask_True - Result: 0.00208
INFO - real_catalogs_PROCAT_mini_mask_True - Completed after 0:00:04


S-Rho: 0.0200, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [115]:
# choose a catalog
chosen_catalog_id = '0003leq'

In [116]:
# show correct
show_correct_catalog(
    catalog_id=chosen_catalog_id,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	2502x0fY h: Leeuwenkuil TA' • Chenin Blanc • Shiraz , d: TA' • Chenin Blanc • Shiraz • Lion's Lai
	070eU0fY h: Ribena • Original • Light. 850 ml, d: • Original • Light. 850 ml
	a6a7Z0fY h: Diamond hill 3 LITER. • Chardonnay • Shi, d: 3 LITER. • Chardonnay • Shiraz • Shiraz/
	183cY0fY h: Marabou chokoladebars 35-46 gram. • Fran, d: 35-46 gram. • Fransk Nougat • Marabou Or
	257as0fY h: Lays chips, bugles eller doritos Sour Cr, d: Sour Cream & Onion - BBQ - Salt - • - Or
	0da4f0fY h: Primitivo di manduria • Carlo Sani. Denn, d: • Carlo Sani. Denne Primitivo har været 
	4809E0fY h: Haribo POSE. DET ER BILLIGT. 100-135 gra, d: POSE. DET ER BILLIGT. 100-135 gram. • Sl
	4815F0fY h: Sodavand 150 cl. Flere varianter. + PANT, d: 150 cl. Flere varianter. + PANT. 3. • Ni
	f1a9z0fY h: Bki kaffe 500 gram. • Classic Gold • ABC, d: 500 gram. • Classic Gold • ABC - hele bø
	3672p0fY h: Duyvis nødder Flere varianter 175-225 gr, d: Flere varianter 175-225 gram
	283c80fY h: Nescafé gold STORT GLAS. • Ins

	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?


Use premade function to nicely display a single prediction.

**Models that do not properly ensure permutation invariance will give different prediction on each run** regardless of being put in .eval() mode.

In [117]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
model_ptrnet.eval()
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_ptrnet,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	a6a7Z0fY h: Diamond hill 3 LITER. • Chardonnay • Shi, d: 3 LITER. • Chardonnay • Shiraz • Shiraz/
	 ?NOT_REAL_OFFER?
------------------------
	3956N0fY h: Harpic wc-rens Flere varianter 750 ml. K, d: Flere varianter 750 ml. KÆMPE PARTI
	6429tqfY h: Nordjyder 500. • Frankfurter • Røde knæk, d: 500. • Frankfurter • Røde knækpølser • G
	 ?NOT_REAL_OFFER?
	4815F0fY h: Sodavand 150 cl. Flere varianter. + PANT, d: 150 cl. Flere varianter. + PANT. 3. • Ni
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	00b64qfY h: avokado, squash, forårsløg eller radiser, d: Bundt
	257as0fY h: Lays chips, bugles eller doritos Sour Cr, d: Sour Cream & Onion - BBQ - Salt - • - Or
	067bTqfY h: Knorr • Orientalsk risret • Indisk risre, d: • Orientalsk risret • Indisk risret • Ch
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	fb3cQ0fY h: Libero bleer • Comfort • UP&GO. 38-68 st, d: • Comfort • UP&GO. 38-68 stk. • •. 400. 
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	fed3AqfY h: Påskebryg 33 cl. KYL

## Model DeepSets

In [118]:
from procat_models import DeepSets, DeepSetsPointerCatalogNetwork

In [119]:
# instantiate the model
model_deepsets = DeepSetsPointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_deepsets.float()

DeepSetsPointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): DeepSets(
    (encode): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
      (6): Linear(in_features=64, out_features=128, bias=True)
      (7): ReLU()
      (8): Linear(in_features=128, out_features=128, bias=True)
      (9): ReLU()
      (10): Linear(in_features=128, out_features=128, bias=True)
      (11): ReLU()
      (12): Linear(in_features=128, out_features=64, bias=True)
      (13): ReLU()
    )
    (out): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=128, bias=True)
      (3)

In [120]:
# how many params
count_params(model_deepsets)

The model has 1,101,024 trainable parameters


In [121]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [122]:
# predict
_ = model_deepsets(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [123]:
print(_[1].type())

torch.LongTensor


In [124]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [125]:
# optimizer
optimizer_deepsets = optim.Adam(
    filter(lambda p: p.requires_grad, model_deepsets.parameters()),
    lr=config['learning_rate'])

### Model Training | DeepSets

In [126]:
# train
last_loss_deepsets = train_epochs(
    model_deepsets,
    optimizer_deepsets,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

Epoch 1 / 1: 100%|██████████| 1/1 [00:16<00:00, 16.28s/ batches, avg_loss=5.29856]


In [127]:
last_loss_deepsets.item()

5.298555374145508

### Model Testing | DeepSets
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [128]:
model_deepsets.eval()  # if we had dropout or batchnorm, we must do this

DeepSetsPointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): DeepSets(
    (encode): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): ReLU()
      (6): Linear(in_features=64, out_features=128, bias=True)
      (7): ReLU()
      (8): Linear(in_features=128, out_features=128, bias=True)
      (9): ReLU()
      (10): Linear(in_features=128, out_features=128, bias=True)
      (11): ReLU()
      (12): Linear(in_features=128, out_features=64, bias=True)
      (13): ReLU()
    )
    (out): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=128, bias=True)
      (3)

#### Sacred Experiment | Model DeepSets
We'll want to track the dataset, model params and the result of tests.

In [129]:
# experiment config
current_tested_model = model_deepsets
current_optimizer = optimizer_deepsets
current_last_loss = last_loss_deepsets

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mini_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mini_mask_True - Started run with ID "348"


Model:  <class 'procat_models.DeepSetsPointerCatalogNetwork'>
Result: 0.0054
K-Tau: -0.1922, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mini_mask_True - Result: 0.00542
INFO - real_catalogs_PROCAT_mini_mask_True - Completed after 0:00:05


S-Rho: -0.2861, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [130]:
# choose a catalog
chosen_catalog_id = '0003leq'

Use premade function to nicely display a single prediction.

**Models that do not properly ensure permutation invariance will give different prediction on each run** regardless of being put in .eval() mode.

In [131]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_deepsets,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	36f3y0fY h: Danpo • Frikadeller • Medister. 400 gram, d: • Frikadeller • Medister. 400 gram
	 ?NOT_REAL_OFFER?
	614ebqfY h: Hellmann's mayonnaise • I glas • I tube., d: • I glas • I tube. 225-400 gram
	9937SqfY h: Salater 150-175 gram. GRAASTEN. Mange va, d: 150-175 gram. GRAASTEN. Mange varianter
	 ?NOT_REAL_OFFER?
	0da4f0fY h: Primitivo di manduria • Carlo Sani. Denn, d: • Carlo Sani. Denne Primitivo har været 
	 ?NOT_REAL_OFFER?
	fbff20fY h: Elvital Alle varianter. 200-250 ml. • Sh, d: Alle varianter. 200-250 ml. • Shampoo • 
	d638dqfY h: Frigodan ISKOLD PRIS. • Broccoli • Fine , d: ISKOLD PRIS. • Broccoli • Fine ærter • M
	222fcqfY h: Bbq mørbrad 450 gram. JENSENS, d: 450 gram. JENSENS
	280auqfY h: Kartoffelmos 4 breve. á 125 gram. QUEEN, d: 4 breve. á 125 gram. QUEEN
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	db3fNqfY h: Andelår JULIUS. 160-200 gram. CHOK PRIS, d: JULIUS. 160-200 gram. CHOK PRIS
	29c0BqfY h: 

## Model SetTransformer

In [132]:
from procat_models import SetTransformer, SetTransformerPointerCatalogNetwork

In [133]:
# instantiate the model
model_settrans = SetTransformerPointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_settrans.float()

SetTransformerPointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): SetTransformer(
    (emb): Sequential(
      (0): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
      (1): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
    )
    (enc): Sequential(
      (0): PMA(
        (mab): MAB(
          (fc_q):

In [134]:
# how many params
count_params(model_settrans)

The model has 1,101,472 trainable parameters


In [135]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [136]:
# predict
_ = model_settrans(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [137]:
print(_[1].type())

torch.LongTensor


In [138]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [139]:
# optimizer
optimizer_settrans = optim.Adam(
    filter(lambda p: p.requires_grad, model_settrans.parameters()),
    lr=config['learning_rate'])

### Model Training | Set Transformer

In [140]:
# train
last_loss_settrans = train_epochs(
    model_settrans,
    optimizer_settrans,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

Epoch 1 / 1: 100%|██████████| 1/1 [00:17<00:00, 17.37s/ batches, avg_loss=5.29873]


In [141]:
last_loss_settrans.item()

5.298732757568359

### Model Testing | Set Transformer
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [142]:
model_settrans.eval()  # if we had dropout or batchnorm, we must do this

SetTransformerPointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): SetTransformer(
    (emb): Sequential(
      (0): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
      (1): SAB(
        (mab): MAB(
          (fc_q): Linear(in_features=64, out_features=64, bias=True)
          (fc_k): Linear(in_features=64, out_features=64, bias=True)
          (fc_v): Linear(in_features=64, out_features=64, bias=True)
          (fc_o): Linear(in_features=64, out_features=64, bias=True)
        )
      )
    )
    (enc): Sequential(
      (0): PMA(
        (mab): MAB(
          (fc_q):

#### Sacred Experiment | Model Set Transformer
We'll want to track the dataset, model params and the result of tests.

In [143]:
# experiment config
current_tested_model = model_settrans
current_optimizer = optimizer_settrans
current_last_loss = last_loss_settrans

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mini_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mini_mask_True - Started run with ID "349"


Model:  <class 'procat_models.SetTransformerPointerCatalogNetwork'>
Result: 0.0054
K-Tau: -0.1089, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mini_mask_True - Result: 0.00542
INFO - real_catalogs_PROCAT_mini_mask_True - Completed after 0:00:05


S-Rho: -0.1646, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [144]:
# choose a catalog
chosen_catalog_id = '0003leq'

Use premade function to nicely display a single prediction.

**Models that do not properly ensure permutation invariance will give different prediction on each run** regardless of being put in .eval() mode.

In [145]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_settrans,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	a367w0fY h: Libresse Flere varianter. • Trusseindlæg, d: Flere varianter. • Trusseindlæg • Bind
------------------------
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	ce94wqfY h: Ice cap rejer 125 gram, d: 125 gram
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	c90d30fY h: Steaks Ca. 1000 gram. AF UNGKVÆG. Små fi, d: Ca. 1000 gram. AF UNGKVÆG. Små fine hver
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	a3ddnqfY h: Schulstad • Skovmand - Original - Ekstra, d: • Skovmand - Original - Ekstra grov. - M
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	08cfm0fY h: Danpo hakket kylling 450 gram, d: 450 gram
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
------------------------
------------------------
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	0cbffqfY h: Pålækker 80-90 gram. - Classic - Tomat. , d: 80-90 gram. - Classic - Tomat. VILDT BIL
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
------------------------
	 

## Model Custom

In [146]:
from procat_models import CustomAttentionSetLayer, CustomAttentionSetEmbedder, CustomAttentionPointerCatalogNetwork

In [147]:
# instantiate the model
model_custom = CustomAttentionPointerCatalogNetwork(
    word_embedding_dim=config['ptr_word_emb_dim'],
    offer_embedding_dim=config['ptr_offer_emb_dim'],
    hidden_dim=config['ptr_hid_dim'],
    offer_rnn_layers=config['ptr_offer_rnn_layers'],
    catalog_rnn_layers=config['ptr_catalog_rnn_layers'],
    dropout_offers=config['ptr_dropout_offers'],
    dropout_catalogs=config['ptr_dropout_catalogs'],
    bidir_offers=config['ptr_bidir_offers'],  # not handled 2021 04
    bidir_catalogs=config['ptr_bidir_catalogs'],
    masking=config['ptr_mask'],
    vocab_size=len(word2idx),
)

model_custom.float()

CustomAttentionPointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): CustomAttentionSetEmbedder(
    (l1): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
      (s): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=1, bias=True)
        (3): ReLU()
      )
      (a): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
    )
    (l2): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bias

In [148]:
# how many params
count_params(model_custom)

The model has 1,097,508 trainable parameters


In [149]:
# 2-elem batch
batch_x = X_train_torch[:2]

In [150]:
# predict
_ = model_custom(batch_x)
print(_[0].size(), _[1].size())

torch.Size([2, 200, 200]) torch.Size([2, 200])


In [151]:
print(_[1].type())

torch.LongTensor


In [152]:
# loss
CCE = torch.nn.CrossEntropyLoss()

In [153]:
# optimizer
optimizer_custom = optim.Adam(
    filter(lambda p: p.requires_grad, model_custom.parameters()),
    lr=config['learning_rate'])

### Model Training | Custom

In [154]:
# train
last_loss_custom = train_epochs(
    model_custom,
    optimizer_custom,
    CCE,
    train_dataloader,
    config['num_epochs'],
    allow_gpu=True,
    x_name=0,
    y_name=1)

Epoch 1 / 1: 100%|██████████| 1/1 [00:16<00:00, 16.24s/ batches, avg_loss=5.29863]


In [155]:
last_loss_custom.item()

5.298625946044922

### Model Testing | Custom
Put the model in eval() mode first (prevents dropout and such from predicting sth else from the same permuted x).

In [156]:
model_custom.eval()  # if we had dropout or batchnorm, we must do this

CustomAttentionPointerCatalogNetwork(
  (word_embedding): Embedding(26169, 32)
  (offer_embedding): PointerOfferEmbedder(
    (offer_rnn): GRU(32, 64, num_layers=2, batch_first=True, dropout=0.05)
  )
  (set_embedding): CustomAttentionSetEmbedder(
    (l1): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
      (s): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=1, bias=True)
        (3): ReLU()
      )
      (a): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): ReLU()
      )
    )
    (l2): CustomAttentionSetLayer(
      (e): Sequential(
        (0): Linear(in_features=64, out_features=64, bias

#### Sacred Experiment | Model Custom
We'll want to track the dataset, model params and the result of tests.

In [157]:
# experiment config
current_tested_model = model_custom
current_optimizer = optimizer_custom
current_last_loss = last_loss_custom

config_update = {
    'model': current_tested_model.__class__.__name__,
    'final_training_loss': round(current_last_loss.item(), 10),
    'optimizer': current_optimizer.__class__.__name__,
    'cfg': config,
}

# run the experiment, with updated config
run_info = ex.run(config_updates=config_update)

INFO - real_catalogs_PROCAT_mini_mask_True - Running command 'run_model_tests'
INFO - real_catalogs_PROCAT_mini_mask_True - Started run with ID "350"


Model:  <class 'procat_models.CustomAttentionPointerCatalogNetwork'>
Result: 0.0071
K-Tau: 0.0145, perc_valid: 100.0


INFO - real_catalogs_PROCAT_mini_mask_True - Result: 0.00708
INFO - real_catalogs_PROCAT_mini_mask_True - Completed after 0:00:05


S-Rho: 0.0214, perc_valid: 100.0


#### Show predictions
See what it predicts.

##### Specific catalog prediction, with offer text features

In [158]:
# choose a catalog
chosen_catalog_id = '0003leq'

In [160]:
# show correct
show_correct_catalog(
    catalog_id=chosen_catalog_id,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	2502x0fY h: Leeuwenkuil TA' • Chenin Blanc • Shiraz , d: TA' • Chenin Blanc • Shiraz • Lion's Lai
	070eU0fY h: Ribena • Original • Light. 850 ml, d: • Original • Light. 850 ml
	a6a7Z0fY h: Diamond hill 3 LITER. • Chardonnay • Shi, d: 3 LITER. • Chardonnay • Shiraz • Shiraz/
	183cY0fY h: Marabou chokoladebars 35-46 gram. • Fran, d: 35-46 gram. • Fransk Nougat • Marabou Or
	257as0fY h: Lays chips, bugles eller doritos Sour Cr, d: Sour Cream & Onion - BBQ - Salt - • - Or
	0da4f0fY h: Primitivo di manduria • Carlo Sani. Denn, d: • Carlo Sani. Denne Primitivo har været 
	4809E0fY h: Haribo POSE. DET ER BILLIGT. 100-135 gra, d: POSE. DET ER BILLIGT. 100-135 gram. • Sl
	4815F0fY h: Sodavand 150 cl. Flere varianter. + PANT, d: 150 cl. Flere varianter. + PANT. 3. • Ni
	f1a9z0fY h: Bki kaffe 500 gram. • Classic Gold • ABC, d: 500 gram. • Classic Gold • ABC - hele bø
	3672p0fY h: Duyvis nødder Flere varianter 175-225 gr, d: Flere varianter 175-225 gram
	283c80fY h: Nescafé gold STORT GLAS. • Ins

In [159]:
# show predicted
# if the model is not permutation invariant, it will predict sth else
# for every random permutation of the source catalog
show_predicted_catalog(
    catalog_id='0003leq',
    a_model=model_custom,
    catalogs_dataframe=df_catalogs,
    offers_dataframe=df)

	1864JqfY h: Santa maria nudler 250 gram. VILDT BILLI, d: 250 gram. VILDT BILLIGT
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	7c1eeqfY h: Kokosmælk 17-19%. 400 ml, d: 17-19%. 400 ml
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	7d7a10fY h: Nakkefilet DANSK. 1/2 KG. Skæres til nak, d: DANSK. 1/2 KG. Skæres til nakkekotelette
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	f4f4MqfY h: Tulip PAKKE. 200 gram. MIDDAGS- FRIKADEL, d: PAKKE. 200 gram. MIDDAGS- FRIKADELLER
	f1a9z0fY h: Bki kaffe 500 gram. • Classic Gold • ABC, d: 500 gram. • Classic Gold • ABC - hele bø
	 ?NOT_REAL_OFFER?
	 ?NOT_REAL_OFFER?
	9937SqfY h: Salater 150-175 gram. GRAASTEN. Mange va, d: 150-175 gram. GRAASTEN. Mange varianter
	 ?NOT_REAL_OFFER?
	708e7qfY h: Cultura drik 500 ml. • Blåbær • Jordbær, d: 500 ml. • Blåbær • Jordbær
	 ?NOT_REAL_OFFER?
	db9dGqfY h: God morgen juice Appelsin/blodgrape Appe, d: Appelsin/blodgrape Appelsin. LITER. FIND
	d4f0v0fY h: Zendium FRIT VALG. • Tandp